# Lê Hoàng Thịnh
## 19520285
Xây dựng mạng neural để giải quyết bài toán phân lớp nhị phân.

Steps to resolve
1. Đọc dữ liệu từ file CSV ( nên lọc ra những cột không có lặp lại -> không ảnh hưởng đến kết quả mô hình như RowNumber, ID, name ...)
2. Chuyển đổi dưới dạng "không thứ tự" (rời rạc) → one hot encoding 
3. Chia 2 tập train và test 
4. Chuẩn hoá tập train (X) sử dụng Standard scaler 
5. Build mạng neural
6. Input → 6 neural / Relu → 6 neural / Relu → 1 neuron / sigmoid
7. Huấn luyện với thuật toán Adam và hàm độ lỗi Binary Cross Entropy
8. Test và đánh giá độ chính xác của mô hình

# Import libraries

In [1]:
import pandas as pd
import numpy as np

1. Read file csv

In [2]:
rawData= pd.read_csv('/content/Churn_Modelling.csv')
rawData.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


2. Remove uneccessary column ( first 3 columns)

In [3]:
data=rawData.drop(['RowNumber','CustomerId','Surname'],axis=1)
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


3. Get the features and output

In [4]:
x= data.iloc[:,0:10]
y= data.iloc[:,10]

# Chuyển đổi dưới dạng "không thứ tự" (rời rạc) → one hot encoding

Convert both Gender and Geography to one-hot encoding

In [5]:
x=pd.get_dummies(x,drop_first=True)
x

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_Germany,Geography_Spain,Gender_Male
0,619,42,2,0.00,1,1,1,101348.88,0,0,0
1,608,41,1,83807.86,1,0,1,112542.58,0,1,0
2,502,42,8,159660.80,3,1,0,113931.57,0,0,0
3,699,39,1,0.00,2,0,0,93826.63,0,0,0
4,850,43,2,125510.82,1,1,1,79084.10,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,39,5,0.00,2,1,0,96270.64,0,0,1
9996,516,35,10,57369.61,1,1,1,101699.77,0,0,1
9997,709,36,7,0.00,1,0,1,42085.58,0,0,0
9998,772,42,3,75075.31,2,1,0,92888.52,1,0,1


# Split to training and testing set

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)

# Using standard scaler to standardize training set

In [8]:
from sklearn.preprocessing import StandardScaler

standard_scaler=StandardScaler()
x_train=standard_scaler.fit_transform(x_train)
x_test= standard_scaler.transform(x_test)

# Build neural networking with 2 layers 
Input → 6 neural / Relu → 6 neural / Relu → 1 neuron / sigmoid

In [9]:
from keras.models import Sequential
from keras.layers import Dense

In [10]:
def build_keras_model():  
  # define the keras model
  model = Sequential()

  # Enter 2 hiddens layers, each has 6 nodes and use relu activation. The output has 1 neuron ( 0,1) use sigmoid activation
  model.add(Dense(6, input_dim=11, activation='relu'))
  model.add(Dense(6, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))

  # compile model
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

  model.summary()

  return model

Fitting model

In [11]:
model=build_keras_model()
model.fit(x_train,y_train,batch_size=10,epochs=50)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 6)                 72        
                                                                 
 dense_1 (Dense)             (None, 6)                 42        
                                                                 
 dense_2 (Dense)             (None, 1)                 7         
                                                                 
Total params: 121
Trainable params: 121
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
800/800 [==============================] - 2s 1ms/step - loss: 0.5301 - accuracy: 0.7341
Epoch 2/50
800/800 [==============================] - 1s 1ms/step - loss: 0.4331 - accuracy: 0.8029
Epoch 3/50
800/800 [==============================] - 1s 1ms/step - loss: 0.4122 - accuracy: 0.8165
Epoch 4/50
800/800 [==

Calculate accuracy

In [12]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression

kfold=KFold(n_splits=5)
logisticRegression= LogisticRegression()

scores=cross_val_score(logisticRegression,x,y,cv=kfold,n_jobs=-1)
print(f'KFold with logistic regression: {np.mean(scores)}, ({np.std(scores)})')

KFold with logistic regression: 0.7901, (0.004892851929090038)


# Testing with input
Thử nghiệm xem với khách hàng có thông tin sau thì có nên exit hay ko?
  Geography: France
  Credit Score: 600
  Gender: Male
  Age: 40 years old
  Tenure: 3 years
  Balance: 60000
  No.of Products: 2
  Has credit card: Yes
  Active Member: Yes
  Estimated Salary: 50000

In [13]:
new_prediction = model.predict(standard_scaler.transform(np.array([[600, 40, 3, 60000, 2, 1, 1, 85000, 0,0,0]])))
print(new_prediction)

[[0.0667052]]


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"


# Accuracy

In [14]:
keras_classifier = KerasClassifier(build_fn = build_keras_model, batch_size = 10, nb_epoch = 50)
accuracies = cross_val_score(estimator = keras_classifier, X = x_train, y = y_train, cv = 10, n_jobs = -1)
print(f'Accuracy {np.mean(accuracies)}')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:705: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


Accuracy 0.7961250007152557
